In [87]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [88]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [89]:
tf.keras.backend.clear_session()

In [90]:
subject = 'Distrito Federal - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [91]:
data = pd.read_csv('2003_mo_model_input_DF.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t),Distrito Federal - Consumo de Cimento (t)
0,2003-1,0.254227,8.293882,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.826971,1.184017e+08,3.570616e+06,43.639430,1.092465e+08,156.625720,50.047
1,2003-2,0.252655,8.287887,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.827063,1.185511e+08,3.573184e+06,43.651300,1.093025e+08,157.678528,48.110
2,2003-3,0.250972,8.281892,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.827155,1.187006e+08,3.575752e+06,43.663170,1.093585e+08,159.190268,49.006
3,2003-4,0.249179,8.275896,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.827247,1.188500e+08,3.578320e+06,43.675041,1.094145e+08,160.688376,48.445
4,2003-5,0.247070,8.269901,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.827340,1.189994e+08,3.580889e+06,43.686911,1.094705e+08,162.847410,51.436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,0.529279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.077918,77.448
236,2022-9,0.527896,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.545664,64.321
237,2022-10,0.526069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,248.265413,72.839
238,2022-11,0.523943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247.486640,59.598


In [92]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [93]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Distrito Federal - Consumo de Cimento (t), Length: 240, dtype: float64

In [94]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
0,-0.765895,-0.830412,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.593530,-1.693822,0.396337,-2.171734,-2.205731,-1.931775
1,-0.800114,-0.833674,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.541888,-1.674984,0.431282,-2.081307,-2.163694,-1.908347
2,-0.836745,-0.836936,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.490246,-1.656145,0.466227,-1.990880,-2.121657,-1.874707
3,-0.875764,-0.840198,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-2.438604,-1.637307,0.501171,-1.900453,-2.079620,-1.841370
4,-0.921678,-0.843461,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-2.386962,-1.618468,0.536116,-1.810026,-2.037583,-1.793327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.937260,1.309227,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.616226,1.235244,-1.506889,-0.747477,1.174551,0.131039
158,0.925760,1.302282,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.563561,1.236423,-1.511843,-0.763990,1.168128,0.077457
159,0.912813,1.295337,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.510895,1.237602,-1.516798,-0.780503,1.161706,0.032090
160,0.906620,1.288392,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.458230,1.238780,-1.521752,-0.797016,1.155283,-0.032489


In [95]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      46.442
1      43.234
2      54.587
3      59.228
4      63.997
        ...  
157    35.599
158    49.981
159    39.881
160    48.598
161    49.942
Name: Distrito Federal - Consumo de Cimento (t), Length: 162, dtype: float64

In [96]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [97]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [98]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Distrito Federal - value,Distrito Federal - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Distrito Federal - IDH,Distrito Federal - PIB - Estadual,Distrito Federal - PIB - Construção Civil,Distrito Federal - PIB - Per Capita,Distrito Federal - PIB - Preços de Mercado,Distrito Federal - Produção de Cimento (t)
126,0.746963,0.952687,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.686090,0.806860,-0.638912,-0.325093,0.894339,1.712291
127,0.759387,0.973455,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.670518,0.828599,-0.679356,-0.322764,0.917121,1.675940
128,0.775249,0.994223,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.654947,0.850339,-0.719800,-0.320434,0.939903,1.626852
129,0.795874,1.014990,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.639375,0.872078,-0.760244,-0.318105,0.962685,1.569267
130,0.815598,1.035758,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.623804,0.893818,-0.800689,-0.315775,0.985467,1.514446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.298634,1.198041,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,0.236110,1.116166,-1.071950,-1.286229,0.817896,-0.512069
188,1.354774,1.198255,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,0.183845,1.104850,-1.047047,-1.328476,0.799100,-0.500824
189,1.401307,1.198470,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,0.131580,1.093534,-1.022144,-1.370723,0.780305,-0.491677
190,1.460103,1.198684,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,0.079315,1.082218,-0.997242,-1.412970,0.761510,-0.481495


In [99]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [100]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [101]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 9)
#     target,target_val = validation_splitter(train_target, 9)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(180, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [102]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [103]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[817943547, 2906557025, 445077704, 2244009983, 3012787981, 456809708, 521500636, 659429028, 528241868, 423278361, 3134708917, 2739007725, 1017807070, 1027587596, 2248497795, 3848838939, 345672479, 405855235, 401583285, 1824731265, 3078448465, 1249594019, 1096020684, 4255926456, 1592554518, 3559698399, 1783395378, 1961485406, 2097741708, 4290938273, 1499629638, 1913973600, 1883298464, 490903925, 2063797277, 3834420398, 1087818063, 3930115184, 2719049953, 2334590867, 490016101, 4224330493, 2866514016, 1422248659, 3371226212, 1562301077, 1842377960, 4237057586, 3124676861, 2483831429]


Step: 0 ___________________________________________
val_loss: 19.813812255859375
winner_seed: 817943547


Step: 1 ___________________________________________
val_loss: 140.46983337402344


Step: 2 ___________________________________________
val_loss: 21.054851531982422


Step: 3 ___________________________________________
val_loss: 27.82184600830078


Step: 4 ___________________________________________
val

In [104]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 130ms/step - loss: 6496.3379 - val_loss: 23.6947
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 1035.2225 - val_loss: 1047.6290
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 361.9496 - val_loss: 1458.6978
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 407.2618 - val_loss: 2410.1604
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 912.2489 - val_loss: 799.8702
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 534.9561 - val_loss: 1117.2872
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 323.2179 - val_loss: 1509.5021
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 267.5386 - val_loss: 327.2816
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 200.1414 - val_loss: 160.0162
Epoch 10/10000
4/4 [==============================] - 0s 7ms/step

Epoch 80/10000
4/4 [==============================] - 0s 8ms/step - loss: 107.4795 - val_loss: 150.3761
Epoch 81/10000
4/4 [==============================] - 0s 8ms/step - loss: 103.7227 - val_loss: 124.8827
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 113.8927 - val_loss: 98.1749
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 106.5110 - val_loss: 130.0124
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 105.1590 - val_loss: 135.1997
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 109.0319 - val_loss: 149.5316
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 115.6444 - val_loss: 96.2177
Epoch 87/10000
4/4 [==============================] - 0s 7ms/step - loss: 125.2817 - val_loss: 110.0625
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 115.5590 - val_loss: 103.6878
Epoch 89/10000
4/4 [==============================] - 0s 7ms/step 

Epoch 159/10000
4/4 [==============================] - 0s 7ms/step - loss: 95.1322 - val_loss: 83.3977
Epoch 160/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.0090 - val_loss: 31.6857
Epoch 161/10000
4/4 [==============================] - 0s 8ms/step - loss: 85.8175 - val_loss: 50.6866
Epoch 162/10000
4/4 [==============================] - 0s 8ms/step - loss: 89.3637 - val_loss: 53.8538
Epoch 163/10000
4/4 [==============================] - 0s 8ms/step - loss: 86.9064 - val_loss: 32.7948
Epoch 164/10000
4/4 [==============================] - 0s 8ms/step - loss: 86.0838 - val_loss: 74.5695
Epoch 165/10000
4/4 [==============================] - 0s 8ms/step - loss: 91.5869 - val_loss: 54.8941
Epoch 166/10000
4/4 [==============================] - 0s 8ms/step - loss: 87.3092 - val_loss: 40.0682
Epoch 167/10000
4/4 [==============================] - 0s 8ms/step - loss: 96.3266 - val_loss: 44.7725
Epoch 168/10000
4/4 [==============================] - 0s 8ms/step - loss

4/4 [==============================] - 0s 7ms/step - loss: 91.7665 - val_loss: 84.4586
Epoch 239/10000
4/4 [==============================] - 0s 7ms/step - loss: 92.9213 - val_loss: 70.7906
Epoch 240/10000
4/4 [==============================] - 0s 8ms/step - loss: 92.6265 - val_loss: 68.1355
Epoch 241/10000
4/4 [==============================] - 0s 8ms/step - loss: 82.6372 - val_loss: 36.8499
Epoch 242/10000
4/4 [==============================] - 0s 7ms/step - loss: 95.7820 - val_loss: 28.4480
Epoch 243/10000
4/4 [==============================] - 0s 7ms/step - loss: 82.9424 - val_loss: 47.0470
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss: 80.0483 - val_loss: 42.7124
Epoch 245/10000
4/4 [==============================] - 0s 7ms/step - loss: 77.4070 - val_loss: 35.2414
Epoch 246/10000
4/4 [==============================] - 0s 7ms/step - loss: 93.8263 - val_loss: 35.2223
Epoch 247/10000
4/4 [==============================] - 0s 7ms/step - loss: 80.4730 - val_

4/4 [==============================] - 0s 8ms/step - loss: 76.4714 - val_loss: 63.3967
Epoch 318/10000
4/4 [==============================] - 0s 8ms/step - loss: 81.9177 - val_loss: 49.5727
Epoch 319/10000
4/4 [==============================] - 0s 8ms/step - loss: 91.4583 - val_loss: 48.6751
Epoch 320/10000
4/4 [==============================] - 0s 7ms/step - loss: 79.1143 - val_loss: 39.9775
Epoch 321/10000
4/4 [==============================] - 0s 7ms/step - loss: 73.4426 - val_loss: 38.4622
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 80.4401 - val_loss: 31.7815
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 70.6972 - val_loss: 30.5531
Epoch 324/10000
4/4 [==============================] - 0s 7ms/step - loss: 76.9745 - val_loss: 43.6891
Epoch 325/10000
4/4 [==============================] - 0s 7ms/step - loss: 84.7050 - val_loss: 48.1792
Epoch 326/10000
4/4 [==============================] - 0s 7ms/step - loss: 67.1024 - val_

...

4/4 [==============================] - 0s 8ms/step - loss: 15.6680 - val_loss: 35.8496
Epoch 871/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.8867 - val_loss: 31.4759
Epoch 872/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.5627 - val_loss: 37.8437
Epoch 873/10000
4/4 [==============================] - 0s 7ms/step - loss: 16.8215 - val_loss: 28.4804
Epoch 874/10000
4/4 [==============================] - 0s 8ms/step - loss: 20.6701 - val_loss: 32.4787
Epoch 875/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.9258 - val_loss: 34.2843
Epoch 876/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.6362 - val_loss: 38.7367
Epoch 877/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.9249 - val_loss: 36.0752
Epoch 878/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1768 - val_loss: 33.6197
Epoch 879/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.4477 - val_

4/4 [==============================] - 0s 9ms/step - loss: 14.4722 - val_loss: 31.6126
Epoch 950/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.4639 - val_loss: 31.0715
Epoch 951/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.7706 - val_loss: 38.4126
Epoch 952/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.7913 - val_loss: 33.7872
Epoch 953/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.6837 - val_loss: 32.4377
Epoch 954/10000
4/4 [==============================] - 0s 7ms/step - loss: 17.4772 - val_loss: 31.9499
Epoch 955/10000
4/4 [==============================] - 0s 7ms/step - loss: 14.6037 - val_loss: 31.7342
Epoch 956/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.3727 - val_loss: 32.3739
Epoch 957/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.6996 - val_loss: 30.2065
Epoch 958/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.7993 - val_

4/4 [==============================] - 0s 8ms/step - loss: 12.3233 - val_loss: 33.4704
Epoch 1029/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.4255 - val_loss: 31.8317
Epoch 1030/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.8865 - val_loss: 32.5918
Epoch 1031/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.9152 - val_loss: 32.0684
Epoch 1032/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.1520 - val_loss: 32.4660
Epoch 1033/10000
4/4 [==============================] - 0s 8ms/step - loss: 14.2533 - val_loss: 32.7862
Epoch 1034/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.9533 - val_loss: 33.7804
Epoch 1035/10000
4/4 [==============================] - 0s 8ms/step - loss: 13.6918 - val_loss: 34.1466
Epoch 1036/10000
4/4 [==============================] - 0s 8ms/step - loss: 12.1255 - val_loss: 33.4439
Epoch 1037/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.49

Epoch 1107/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.1609 - val_loss: 31.0473
Epoch 1108/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.6702 - val_loss: 31.7404
Epoch 1109/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.5465 - val_loss: 31.5931
Epoch 1110/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.5709 - val_loss: 34.2548
Epoch 1111/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.6330 - val_loss: 31.6444
Epoch 1112/10000
4/4 [==============================] - 0s 7ms/step - loss: 13.7688 - val_loss: 30.3822
Epoch 1113/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.1056 - val_loss: 32.5012
Epoch 1114/10000
4/4 [==============================] - 0s 7ms/step - loss: 9.7727 - val_loss: 33.2439
Epoch 1115/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.3980 - val_loss: 30.5492
Epoch 1116/10000
4/4 [==============================] - 0s 8ms/st

Epoch 1186/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.2031 - val_loss: 34.5891
Epoch 1187/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.8460 - val_loss: 30.6228
Epoch 1188/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.1509 - val_loss: 32.0985
Epoch 1189/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.2020 - val_loss: 32.1999
Epoch 1190/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.8605 - val_loss: 32.2866
Epoch 1191/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.7294 - val_loss: 31.2376
Epoch 1192/10000
4/4 [==============================] - 0s 7ms/step - loss: 11.1081 - val_loss: 33.1247
Epoch 1193/10000
4/4 [==============================] - 0s 7ms/step - loss: 10.1944 - val_loss: 31.9296
Epoch 1194/10000
4/4 [==============================] - 0s 7ms/step - loss: 12.1400 - val_loss: 30.5891
Epoch 1195/10000
4/4 [==============================] - 0s 7ms/s

In [105]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [106]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,45.833691,45.881226,45.871128,45.886814,45.873013,45.909546,45.930824,46.013889,45.956223,45.974556,46.00737,46.14761,45.900936,46.225437,46.401173,46.192799,46.302208,46.535973,46.530994,47.240822,47.337921,49.472263,51.153061,51.199509,51.262581,51.139694,51.141106,47.461563,51.048008,48.689045
Target,53.235,59.29,54.875,57.52,49.573,40.698,47.195,43.527,45.488,44.96,36.721,54.463,51.462,58.854,53.94,51.514,42.635,40.371,48.99,44.726,44.491,43.901,55.019,50.942,63.992,64.127,60.822,65.182,51.282,43.554
Error,7.40131,13.408775,9.003872,11.633186,3.699989,5.211544,1.264175,2.486889,0.468224,1.014557,9.286369,8.315392,5.561066,12.628563,7.538826,5.321201,3.66721,6.164974,2.459007,2.51482,2.84692,5.571262,3.86594,0.257507,12.72942,12.987305,9.680893,17.720436,0.233994,5.135044


In [107]:
display(mae)
display(mape)

6.335955

0.119618155

In [108]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [109]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined